In [1]:
import pandas as pd
import numpy as np

In [10]:
subject = 1
setsize = 36
# Re-establish the file path and reload the CSV file into a DataFrame
file_path = f'../data/subject_files/sub-{subject}_setsize-{setsize}_desc-gazes.csv'
df = pd.read_csv(file_path)


In [3]:

# Function to map 'item' values to (x, y) coordinates in a 6x6 grid
def map_item_to_coordinates(item):
    x = (item - 1) % 6  # X-coordinate (column)
    y = (item - 1) // 6  # Y-coordinate (row)
    return x, y


In [4]:
# Reload the CSV file into a DataFrame now that it's been re-uploaded
# df = pd.read_csv('/mnt/data/sub-1_setsize-36_desc-gazes.csv')

# Initialize a dictionary to hold the fixation sequences for each trial
fixation_sequences = {}

# Group the DataFrame by 'trial'
for trial, group_df in df.groupby('trial'):
    # Initialize a list to hold the fixation sequence for this trial
    fixation_sequence = []
    
    # Iterate over each row in the group DataFrame
    for _, row in group_df.iterrows():
        # Map the 'item' value to (x, y) coordinates
        x, y = map_item_to_coordinates(row['item'])
        
        # Get the duration ('dur') of the fixation
        duration = row['dur']
        
        # Append this fixation to the fixation sequence for this trial
        fixation_sequence.append({'x': x, 'y': y, 'duration': duration})
    
    # Save this fixation sequence to the fixation_sequences dictionary
    fixation_sequences[trial] = fixation_sequence

# Display a sample fixation sequence for one trial (for demonstration)
sample_trial = list(fixation_sequences.keys())[0]
fixation_sequences[sample_trial][:5]  # Displaying only first 5 fixations for sample trial


[{'x': 2, 'y': 2, 'duration': 212.0},
 {'x': 2, 'y': 1, 'duration': 26.0},
 {'x': 1, 'y': 0, 'duration': 300.0},
 {'x': 1, 'y': 1, 'duration': 186.0},
 {'x': 4, 'y': 4, 'duration': 226.0}]

In [5]:
# Define the sample time in milliseconds based on the 500Hz sampling rate
# 1 second = 1000 milliseconds, so 1/500 * 1000 gives the sample time in milliseconds
sample_time_ms = 1 / 500 * 1000

# Standardize the duration into ticks based on the sample time
for trial, fixations in fixation_sequences.items():
    for fixation in fixations:
        fixation['ticks'] = int(round(fixation['duration'] / sample_time_ms))

# Display the standardized ticks for the sample trial
fixation_sequences[sample_trial][:5]  # Displaying only first 5 fixations for the sample trial


[{'x': 2, 'y': 2, 'duration': 212.0, 'ticks': 106},
 {'x': 2, 'y': 1, 'duration': 26.0, 'ticks': 13},
 {'x': 1, 'y': 0, 'duration': 300.0, 'ticks': 150},
 {'x': 1, 'y': 1, 'duration': 186.0, 'ticks': 93},
 {'x': 4, 'y': 4, 'duration': 226.0, 'ticks': 113}]

In [7]:
import os


In [13]:

# Create a directory to store the output files, if it doesn't exist
fixation_output_dir = '../data/fixation_sequences'
os.makedirs(fixation_output_dir, exist_ok=True)

# Define the sample time in milliseconds based on the 500Hz sampling rate
# 1 second = 1000 milliseconds, so 1/500 * 1000 gives the sample time in milliseconds
sample_time_ms = 1 / 500 * 1000


# Iterate through each trial's fixation sequence
for trial, fixations in fixation_sequences.items():
    simplified_sequence = []
    
    # Iterate over each fixation in the sequence
    for fixation in fixations:
        x, y = fixation['x'], fixation['y']
        duration = fixation['duration']
        
        # Convert the duration into ticks based on the sample time
        ticks = int(round(duration / sample_time_ms))
        
        # Append this x, y pair consecutively for the number of ticks/10
        ticks //= 10  # Conserve space by dividing the number of ticks by 10
        simplified_sequence.extend([(x, y)] * ticks)
        
    # Convert the simplified sequence to a string representation
    sequence_str = 'OUTPUT/2\n' + '\n'.join([f"{x} {y}" for x, y in simplified_sequence])
    
    # Create the output file path
    output_file_path = os.path.join(fixation_output_dir, f"s_{subject}_{trial}_fixation_sequence.dat")
    
    # Write the simplified sequence to a .dat file
    with open(output_file_path, 'w') as f:
        f.write(sequence_str)
